In [ ]:
import pandas as pd
import difflib
import numpy as np

In [ ]:
yt = pd.read_csv("Youtube_url_views_final.csv")
yt.drop(['Unnamed: 0' , 'Unnamed: 0.1'] , axis=1, inplace=True)

# Completezza

Contiamo quanti valori nulli ci sono dopo la raccolta dei dati da youtube

In [ ]:
yt.isnull().sum()

# Accuratezza

Controlliamo che il video preso con lo scraping corrisponda a quello della canzone di Spotify

In [ ]:
yt['Title'] = yt['Title'].fillna("Null")

Prima controlliamo che il nome della canzone (``Track``) sia abbastanza simile al titolo del video su youtube (``Title``)

In [ ]:
def string_similarity(x):
    if x['Title'] == 'Null':
        return None
    else:
        return difflib.SequenceMatcher(None, x['Track'], x['Title']).ratio()
        

yt['similarity'] = yt.apply(lambda x: string_similarity(x), axis=1)

Scelta la soglia di 0.3, aggiungiamo un altro controllo per le canzoni al di sotto di questa soglia: consideriamo un video giusto se l'artista è contenuto nel nome del canale che ha pubblicato il video, se è contenuto nel titolo del video, o se il nome della canzone è contenuta nel titolo del video


In [ ]:
def check(x):
    if x['similarity'] < 0.3:
        if (x['Artist'].lower() in x['Title'].lower())|(x['Artist'].lower() in x['Channel'].lower())|(x['Track'].lower() in x['Title'].lower()):
            return True
        else:
            return False

def true_video(x):
    if (x['similarity'] >= 0.3):
        return True
    else:
        if x['check'] == True:
            return True
        else:
            return False
    
    
yt['check'] = yt.apply(lambda x: check(x), axis=1)
yt['True_Video'] = yt.apply(lambda x: true_video(x), axis=1)

A questo punto per le righe che non consideriamo come video accurato rispetto alla canzone di Spotify impostiamo un valore nullo per gli attributi raccolti da youtube

In [ ]:
yt.drop(['similarity' , 'check'], axis=1, inplace=True)

In [ ]:
mask = yt['True_Video'] == False
yt.loc[mask, ['Url', 'Title', 'Channel', 'Views', 'Likes','Comments', 'Description', 'Licensed']] = None

In [ ]:
yt.drop(['True_Video'], axis=1, inplace=True)

### Controlliamo ora se la ricerca con YouTube API ha trovato un video ufficiale: 
* se la colonna 'Licensed' è uguale a ``True``, lo prendo per buono
* se la colonna è nulla lascio il campo nullo 
* se la colonna è uguale a ``False`` controllo che il nome del canale sia lo stesso dell'artista o che sia contenuto nel titolo del video

In [ ]:
def official(x):
    video = None
    
    if x['Licensed'] == True:
        video = True
        
    elif x['Licensed'] == False:
        art = x['Artist']
        tit = x['Title']
        chan = x['Channel']
        if (chan in art) | (chan in tit):
            video = True
        else:
            video = False
    
    return video

In [ ]:
yt['official_video'] = yt.apply(official, axis=1)

In [ ]:
yt[yt['official_video']==True].count()[0]

Infine carichiamo tutto in un nuovo file csv

In [ ]:
yt.to_csv("Youtube_Sistemato.csv")